In [1]:
import numpy as np
from dataclasses import dataclass

In [2]:
# First we define class used as storage for N-qbits state.
class NQbit:
    def __init__(self, nqbits=None, state=None):
        """Create storge for qbits. 
        If `nqbits` is specified, creates an empty storage for given number of quantum bits.
        If `state` is given, use the matrix as state vector."""
        if state is None:
            self.nqbits = nqbits
            self.state = np.empty((2,)*nqbits, dtype=np.cdouble)
        else:
            self.state = np.array(state, dtype=np.cdouble)
            self.nqbits = self.state.ndim
            assert self.state.shape == (2,)*self.nqbits
    @staticmethod
    def classical(values=[0]):
        """Constructor for classical state with deterministic values for every qbit."""
        nqbs = NQbit(nqbits=len(values))
        nqbs.state[:] = 0
        nqbs.state[(tuple(values))] = 1
        return nqbs
    @staticmethod
    def random(nbits=1):
        """Constructor for a random state for given number of qbits."""
        real, imag = np.random.randn(*(2,)*nbits), np.random.randn(*(2,)*nbits)
        return NQbit(state=real+1j*imag).normalize()
    @staticmethod
    def bell(nqbits=1):
        """Create Bell state |00>+|11> for given number of qbits."""
        nqbs = NQbit(nqbits=nqbits)
        nqbs.state[:] = 0
        nqbs.state[(0,)*nqbits] = nqbs.state[(1,)*nqbits] = np.power(0.5, 0.5)        
        return nqbs
    def normalize(self):
        """Ensure normalization of the state."""
        nrm = np.linalg.norm(self.state.flatten(), ord=2)
        if nrm>1e-8: self.state /= nrm
        return self
    def tensor_product(self, other):
        """Return tensor product of states. Qbits of self go before qbits of other."""
        nqbs = NQbit(nqbits=self.nqbits+other.nqbits)
        nqbs.state[:] = np.outer(self.state.flatten(), other.state.flatten()).reshape(nqbs.state.shape)
        return nqbs
    def inner_product(self, other):
        return np.inner(np.conj(other.state).flatten(), self.state.flatten())
    def __mul__(self, other):
        """We will treat * operator as inner product of states"""
        return self.inner_product(other)
    def __matmul__(self, other):
        """We will treat @ operator as tensor product of states"""
        return self.tensor_product(other)    
    def __str__(self):
        """Return string representation. Slow. Very slow."""
        it = np.nditer(self.state, flags=['multi_index'])
        dt = {f"|{''.join(map(str, it.multi_index))}>": complex(k) for k in it if np.abs(k)>1e-4}
        return "+".join([f"{v}{k}" for k,v in dt.items()])

a = NQbit.classical([0])
b = NQbit.classical([1])
c = NQbit.bell()
print(f"a = {a}")
print(f"b = {b}")
print(f"c = {c}")
print("a@b =", a@b)
print("a@c =", a@c)
print("c@c =", c*c)

a = (1+0j)|0>
b = (1+0j)|1>
c = (0.7071067811865476+0j)|0>+(0.7071067811865476+0j)|1>
a@b = (1+0j)|01>
a@c = (0.7071067811865476+0j)|00>+(0.7071067811865476+0j)|01>
c@c = (1.0000000000000002+0j)


In [3]:
# Next we define abstract class for gates.
class Gate:
    def __call__(self, state: NQbit):
        """Apply the gate to the state and return modified state."""
        raise NotImplementedError
    def __mul__(self, other):
        """Gates composition will be denoted as *."""
        return Circuit(self, other)
        
# Gates composition = operators composition
class Circuit(Gate):
    def __init__(self, left:Gate, right:Gate):
        self.left = left
        self.right = right
    def __call__(self, state: NQbit):
        return self.left(self.right(state))
    def __str__(self):
        return f"{self.left}{self.right}"
        
# Operator defined by self-adjoint matrix M and list of qbits affected by the matrix.
class MatrixGate(Gate):
    def __init__(self, submatrix=None, bits=[0]):
        self.bits = np.array(bits, dtype=np.int)   
        self._nbits = len(self.bits)
        self.submatrix = np.array(submatrix, dtype=np.cdouble)
        if self.submatrix.ndim==2 and self._nbits>1: # Transorm 2D matrix representation to operator on NDArray
            self.submatrix = self.submatrix.reshape((2,2)*self._nbits)
        assert self.submatrix.shape==(2,2)*self._nbits

    def __call__(self, nqbs: NQbit):
        N = nqbs.nqbits
        in_idx = np.arange(N) # Input state indices
        out_idx = np.arange(N) # Output state indices
        out_idx[self.bits] = np.arange(N, N+self._nbits)
        matrix_indices = tuple(np.arange(N, N+self._nbits))+tuple(self.bits) # First half is where to write result, second half fixes source of data.
        state = np.einsum(self.submatrix, matrix_indices, nqbs.state, in_idx, out_idx)
        return NQbit(state=state)
    
    def __str__(self):
        return f"{self.__class__.__name__}({','.join(map(str,self.bits))})"
    
# Basic gates.
class Identity(Gate):
    def __init__(self):
        pass
    def __call__(self, state:NQbit):
        return state
    def __str__(self):
        return "I"
    
class H(MatrixGate):
    def __init__(self, bit=0):
        x = np.power(0.5, 0.5)
        super().__init__(submatrix=[[x, x], [x, -x]], bits=[bit])

class X(MatrixGate):
    def __init__(self, bit=0):
        super().__init__(submatrix=[[0, 1], [1, 0]], bits=[bit])

class Y(MatrixGate):
    def __init__(self, bit=0):
        super().__init__(submatrix=[[0, -1j], [1j, 0]], bits=[bit])
        
class Z(MatrixGate):
    def __init__(self, bit=0):
        super().__init__(submatrix=[[1, 0], [0, -1]], bits=[bit])

class S(MatrixGate):
    def __init__(self, bit=0):
        super().__init__(submatrix=[[1, 0], [0, 1j]], bits=[bit])
        
class T(MatrixGate):
    def __init__(self, bit=0):
        super().__init__(submatrix=[[1, 0], [0, np.exp(0.25j*np.pi)]], bits=[bit])
        
class CNOT(MatrixGate):
    def __init__(self, bits=[0,1]):
        super().__init__(submatrix=[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], bits=bits)

class CZ(MatrixGate):
    def __init__(self, bits=[0,1]):
        super().__init__(submatrix=[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]], bits=bits)
        
class SWAP(MatrixGate):
    def __init__(self, bits=[0,1]):
        super().__init__(submatrix=[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]], bits=bits)     
        
class CCNOT(MatrixGate):
    def __init__(self, bits=[0,1,2]):
        m = np.eye(8)
        m[-2:,-2:] = [[0,1],[1,0]]
        super().__init__(submatrix=m, bits=bits)
        
# Some unit tests
def almost_equal(x, y, tol=1e-8):
    return np.max(np.abs(x-y))<tol, f"{x} and {y} are not equal"

def test_unitarity(a:Gate, dim:int=None, tests=10):
    for _ in range(tests):
        x, y = NQbit(dim), NQbit(dim)
        assert almost_equal(x*y, a(x)*a(y))
        
test_unitarity(Identity(), dim=3)
test_unitarity(H(), dim=1)
test_unitarity(X(), dim=1)
test_unitarity(Y(), dim=1)
test_unitarity(Z(), dim=1)
test_unitarity(S(), dim=1)
test_unitarity(T(), dim=1)
test_unitarity(CNOT(), dim=2)
test_unitarity(CZ(), dim=2)
test_unitarity(SWAP(), dim=2)
test_unitarity(CCNOT(), dim=3)

# Test basic identities
def almost_equal_qb(x:NQbit, y:NQbit, tol=1e-8):
    return np.max(np.abs(x.state-y.state))<tol

def test_gates_identity(a:Gate, b:Gate, dim=None, tests=10):
    for _ in range(tests):
        x = NQbit(dim)
        assert almost_equal_qb(a(x), b(x)), f"Gates {a} and {b} are not equivalent"
    print(f"{a} == {b}")
        
test_gates_identity(X()*X(), Identity(), dim=1)
test_gates_identity(Y()*Y(), Identity(), dim=1)
test_gates_identity(Z()*Z(), Identity(), dim=1)
test_gates_identity(X()*Y()*Z(), MatrixGate(1j*np.eye(2)), dim=1)
test_gates_identity(S()*S(), Z(), dim=1)
test_gates_identity(T()*T(), S(), dim=1)
test_gates_identity(SWAP()*SWAP(), Identity(), dim=2)
test_gates_identity(CNOT()*CNOT(), Identity(), dim=2)
test_gates_identity(CCNOT()*CCNOT(), Identity(), dim=3)
test_gates_identity(H()*X()*H(), Z(), dim=1)
test_gates_identity(H(1)*CNOT([0,1])*H(1), CZ([0,1]), dim=2)
test_gates_identity(CNOT([1,0])*CNOT([0,1])*CNOT([1,0]), SWAP([0,1]), dim=2)
test_gates_identity(CNOT([0,1])*CNOT([1,0])*CNOT([0,1]), SWAP([0,1]), dim=2)


# Bell state construction
in_state = NQbit.classical([0,0])
circuit = CNOT([0,1])*H(0) # Operators order is inverse to the order of gates in the circuit 
out_state = circuit(in_state)
bell_state = NQbit.bell(nqbits=2)
assert almost_equal_qb(bell_state, out_state)
print(f"{circuit}( {in_state} ) = {out_state}")

X(0)X(0) == I
Y(0)Y(0) == I
Z(0)Z(0) == I
X(0)Y(0)Z(0) == MatrixGate(0)
S(0)S(0) == Z(0)
T(0)T(0) == S(0)
SWAP(0,1)SWAP(0,1) == I
CNOT(0,1)CNOT(0,1) == I
CCNOT(0,1,2)CCNOT(0,1,2) == I
H(0)X(0)H(0) == Z(0)
H(1)CNOT(0,1)H(1) == CZ(0,1)
CNOT(1,0)CNOT(0,1)CNOT(1,0) == SWAP(0,1)
CNOT(0,1)CNOT(1,0)CNOT(0,1) == SWAP(0,1)
CNOT(0,1)H(0)( (1+0j)|00> ) = (0.7071067811865476+0j)|00>+(0.7071067811865476+0j)|11>


In [4]:
# Now we procced to observables and measurements
class Observable:
    def __init__(self, submatrix=None, bits=None):
        self.bits=np.array(bits, dtype=np.int)
        self._nbits = len(self.bits)
        self.submatrix = np.array(submatrix, dtype=np.cdouble)
        assert self.submatrix.ndim==2
        assert self.submatrix.shape==(2**self._nbits,)*2
        assert np.max(np.abs(self.submatrix-self.submatrix.T))<1e-8, "Matrix is not self-adjoint"
        self.evalues, self.evectors = np.linalg.eigh(self.submatrix)
        self.evectors = self.evectors.T.reshape( (-1,)+(2,)*self._nbits )
        self.submatrix = self.submatrix.reshape( (2,2)*self._nbits )
        
    def __call__(self, nqbs:NQbit):
        N = nqbs.nqbits
        in_idx = np.arange(N) # Input state indices
        out_idx = np.arange(N) # Output state indices
        out_idx[self.bits] = np.arange(N, N+self._nbits)
        matrix_indices = tuple(np.arange(N, N+self._nbits))+tuple(self.bits) # First half is where to write result, second half fixes source of data.
        state = np.einsum(self.submatrix, matrix_indices, nqbs.state, in_idx, out_idx)
        return NQbit(state=state)

    def mean(self, state:NQbit) -> float:
        """Return mean value of the observable on the state"""       
        return ((state*self(state))/(state*state)).real

    def measure(self, nqbs:NQbit):
        """Make the measurement and return { value: (probability, collapsed state) }"""
        # Project state on eigenspaces of the observable.
        N = nqbs.nqbits
        in_idx = np.arange(N) # Input state indices
        out_idx = (N+1,)+tuple(k for k in in_idx if k not in self.bits) # Output state indices
        matrix_indices = (N+1,)+tuple(self.bits) 
        projections = np.einsum(self.evectors, matrix_indices, nqbs.state, in_idx, out_idx)
        # Find probability for every eigensubspace.
        probabilities = np.sum( np.abs(projections).reshape(projections.shape[0],-1), axis=1 )
        probabilities /= np.sum(probabilities)
        # Find collapsed states.
        in2_idx = out_idx[1:]
        out2_idx = np.arange(N) 
        mat2_idx = self.bits
        outcomes = []
        for n in range(len(probabilities)):
            p = probabilities[n]
            v = self.evalues[n]
            s = np.einsum(self.evectors[n], mat2_idx, projections[n], in2_idx, out2_idx)
            outcomes.append( Outcome(value=v, p=p, state=NQbit(state=s).normalize()) )
        return PDist(outcomes).merge()
        
@dataclass
class Outcome:
    value: float # value of observable
    p: float # probability
    state: NQbit # collapsed state
    
class PDist: 
    """Probability distribution of observable values."""
    def __init__(self, outcomes):
        self.outcomes = list(outcomes)
        self.p = np.array(list(o.p for o in self.outcomes), dtype=np.float64)
        self.v = np.array(list(o.value for o in self.outcomes), dtype=np.float64)
        assert np.all(self.p>=-1e-8)
        assert np.abs(np.sum(self.p)-1)<1e-8
    def mean(self):
        """Compute mean value"""
        return np.sum(self.p*self.v)
    def var(self):
        """Compute variance"""        
        m = self.mean()
        return np.sum((self.v-m)**2 * self.p)
    def merge(self):
        """Merge outcomes corresponding to the same value of observable."""
        return self
    def __str__(self):
        # Slow, use with caution
        return "\n  " + "\n  ".join(f"{v} p={p} {o.state}" for p, v, o in zip(self.p, self.v, self.outcomes))
    
# Measure single bit
class Bit(Observable):
    """Measurment for single qbit."""
    def __init__(self, bit=0):
        super().__init__(submatrix=[[0,0],[0,1]], bits=[bit])

class PM(Observable):
    """Measure single bit in |0>+|1> and |0>-|1> basis."""
    def __init__(self, bit=0):
        super().__init__(submatrix=[[0,1],[1,0]], bits=[bit])
        
# Tests
x = NQbit.classical([0])@NQbit.bell()@NQbit.classical([1])
print("State:", x)
print("Mean values of bits:", Bit(0).mean(x), Bit(1).mean(x), Bit(2).mean(x))
print("Mean values in |+> |-> basis:", PM(0).mean(x), PM(1).mean(x), PM(2).mean(x))
print("P-Dist 1st bit", Bit(0).measure(x))
print("P-Dist 2nd bit", Bit(1).measure(x))
print("P-Dist 3rd bit", Bit(2).measure(x))

State: (0.7071067811865476+0j)|001>+(0.7071067811865476+0j)|011>
Mean values of bits: 0.0 0.5 1.0
Mean values in |+> |-> basis: 0.0 1.0 0.0
P-Dist 1st bit 
  0.0 p=1.0 (0.7071067811865476+0j)|001>+(0.7071067811865476+0j)|011>
  1.0 p=0.0 
P-Dist 2nd bit 
  0.0 p=0.5 (1+0j)|001>
  1.0 p=0.5 (1+0j)|011>
P-Dist 3rd bit 
  0.0 p=0.0 
  1.0 p=1.0 (0.7071067811865476+0j)|001>+(0.7071067811865476+0j)|011>
